<a href="https://colab.research.google.com/github/alakade-newageproducts/pdf-mini-project/blob/pdf-data-tables-extraction-OCR/pdf_data_tables_extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install -r requirements.txt
# !sudo apt install tesseract-ocr

# import os
# tesseract_path = '/usr/bin'
# os.environ['PATH'] += ':' + tesseract_path

In [ ]:
# from img2table.document import PDF
# from img2table.ocr import TesseractOCR
# import PyPDF2

# pdf_file = open('/content/sample1.pdf', 'rb')
# pdf_reader = PyPDF2.PdfReader(pdf_file)

# num_pages = len(pdf_reader.pages)
# print("Number of pages:", num_pages)

Number of pages: 5


# Extraction of Multiple tables from a pdf file and then selecting appropriate required table out of all tables and finally merged them into one DataFrame.

In [ ]:

import os
from img2table.document import PDF
from img2table.ocr import TesseractOCR

import pandas as pd
import numpy as np

source_file = '/content/sample1.pdf'
pdf_file = open(source_file, 'rb')
pdf_reader = PyPDF2.PdfReader(pdf_file)
num_pages = len(pdf_reader.pages)

# Instantiation of the pdf
pdf = PDF(src=source_file, pages=[0, num_pages-1])

# Instantiation of the OCR, Tesseract, which requires prior installation
ocr = TesseractOCR(lang="eng")

# Table identification and extraction
pdf_tables = pdf.extract_tables(ocr=ocr,borderless_tables=True)

# # We can also create an excel file with the tables
pdf.to_xlsx('tables.xlsx',ocr=ocr)

# Read the Excel file
xlsx = pd.read_excel('tables.xlsx', sheet_name=None)

# Create an empty dictionary to store dataframes with similar sheet names
sheetwise_dfs = {}

# Loop through all sheets in the Excel file
for sheet_name, df in xlsx.items():
    # Check if the sheet name starts with 'Page', which indicates it's a table extracted by img2table
    if sheet_name.startswith('Page'):
        # Get the sheet name prefix, which indicates the PDF page the table was extracted from
        prefix = sheet_name.split(' - ')[0]
        # Check if a dataframe with the same prefix already exists in the dictionary
        if prefix in sheetwise_dfs:
            # If it exists, add the new dataframe to the dictionary
            sheetwise_dfs[prefix].append(df)
        else:
            # If it doesn't exist, create a new list and add the new dataframe to it in the dictionary
            sheetwise_dfs[prefix] = [df]

# Create an empty list to store dataframes for concatenation
final_dfs = []

# Loop through all sheets in the sheetwise dataframes dictionary
for prefix, dfs in sheetwise_dfs.items():
    # Get the maximum number of columns in all dataframes for this sheet prefix
    max_cols = max(df.shape[1] for df in dfs)
    # Select only dataframes with the maximum number of columns for this sheet prefix
    max_col_dfs = [df for df in dfs if df.shape[1] == max_cols]
    # Check if there are any dataframes with the maximum number of columns
    if max_col_dfs:
        # Concatenate all dataframes with the maximum number of columns for this sheet prefix
        merged_df = pd.concat(max_col_dfs, ignore_index=True)
        # Drop columns with all NaN and ' '
        merged_df = merged_df.loc[:, ~(merged_df.isna() | (merged_df == ' ')).all()]
        # Drop rows with only one non-NaN value
        merged_df = merged_df.dropna(thresh=2)
        #Remove UnNamed Columns
        merged_df = merged_df.loc[:, ~merged_df.columns.str.contains('^Unnamed')]
        #Fill NaN with Blank values
        merged_df = merged_df.fillna(' ')
        # Add the concatenated dataframe to the final dataframes list
        final_dfs.append(merged_df)

# Concatenate all dataframes in the final dataframes list into a single dataframe
final_df = pd.concat(final_dfs, ignore_index=True)
final_df.head()

,Exch,Order\nNo.,Order\nTime,Trade\nNo.,Trade\nTime,Security / Contract\ndescription,B/\nS,Quantity,Gross Rate/Trade\nPrice Per Unit\n2 (Rs) @,Brokerage per\nUnit\n(Rs),Net Rate\nper Unit\n(Rs),Closing Rate per\nUnit (only for\nDerivatives)\n(Rs.),Net Total (Before\nLevies)\n(Rs)
0,NSE,1000000007799261.0,10:52:35,1328450.0,10:52:35,ALLCARGO LOGISTICS,B,1,402.9500,1.01,403.9600,0.0000,403.96
1,,,,,,Delivery Total. - Dr.,,1,,,,,403.96
2,NSE,1000000007842251.0,10:53:30,1334682.0,10:53:30,APL APOLLO TUBES LTD,B,3,"1,116.8500",2.79,"1,119.6400",0.0000,"3,358.92"
3,NSE,1000000008028911.0,10:56:54,1361631.0,10:56:54,ISIN :\nINE702C01027,B,5\n5\n1\n1,895.0000,2.24,897.2400,"1,119.6400\n0.0000","5,598.20\n5,598.20\n897.24\n897.24"
4,,,,,,Delivery Total. - Dr.,,4,,,,,490.84


## Same Program with PaddleOCR

In [ ]:

import os
from img2table.document import PDF
from paddleocr import PaddleOCR
import PyPDF2
import pandas as pd
import numpy as np

source_file = '/content/sample1.pdf'
pdf_file = open(source_file, 'rb')
pdf_reader = PyPDF2.PdfReader(pdf_file)
num_pages = len(pdf_reader.pages)

# Instantiation of the pdf
pdf = PDF(src=source_file, pages=[0, num_pages-1])

# Instantiation of the OCR, Tesseract, which requires prior installation
ocr = PaddleOCR(lang="eng")

# Table identification and extraction
pdf_tables = pdf.extract_tables(ocr=ocr,borderless_tables=True)

# # We can also create an excel file with the tables
pdf.to_xlsx('tables.xlsx',ocr=ocr)

# Read the Excel file
xlsx = pd.read_excel('tables.xlsx', sheet_name=None)

# Create an empty dictionary to store dataframes with similar sheet names
sheetwise_dfs = {}

# Loop through all sheets in the Excel file
for sheet_name, df in xlsx.items():
    # Check if the sheet name starts with 'Page', which indicates it's a table extracted by img2table
    if sheet_name.startswith('Page'):
        # Get the sheet name prefix, which indicates the PDF page the table was extracted from
        prefix = sheet_name.split(' - ')[0]
        # Check if a dataframe with the same prefix already exists in the dictionary
        if prefix in sheetwise_dfs:
            # If it exists, add the new dataframe to the dictionary
            sheetwise_dfs[prefix].append(df)
        else:
            # If it doesn't exist, create a new list and add the new dataframe to it in the dictionary
            sheetwise_dfs[prefix] = [df]

# Create an empty list to store dataframes for concatenation
final_dfs = []

# Loop through all sheets in the sheetwise dataframes dictionary
for prefix, dfs in sheetwise_dfs.items():
    # Get the maximum number of columns in all dataframes for this sheet prefix
    max_cols = max(df.shape[1] for df in dfs)
    # Select only dataframes with the maximum number of columns for this sheet prefix
    max_col_dfs = [df for df in dfs if df.shape[1] == max_cols]
    # Check if there are any dataframes with the maximum number of columns
    if max_col_dfs:
        # Concatenate all dataframes with the maximum number of columns for this sheet prefix
        merged_df = pd.concat(max_col_dfs, ignore_index=True)
        # Drop columns with all NaN and ' '
        merged_df = merged_df.loc[:, ~(merged_df.isna() | (merged_df == ' ')).all()]
        # Drop rows with only one non-NaN value
        merged_df = merged_df.dropna(thresh=2)
        #Remove UnNamed Columns
        merged_df = merged_df.loc[:, ~merged_df.columns.str.contains('^Unnamed')]
        #Fill NaN with Blank values
        merged_df = merged_df.fillna(' ')
        # Add the concatenated dataframe to the final dataframes list
        final_dfs.append(merged_df)

# Concatenate all dataframes in the final dataframes list into a single dataframe
final_df = pd.concat(final_dfs, ignore_index=True)
final_df.head()

,Exch,Order\nNo.,Order\nTime,Trade\nNo.,Trade\nTime,Security / Contract\ndescription,B/\nS,Quantity,Gross Rate/\nTrade Price\nPer unit (in\nforeign\n1 currency),Gross Rate/Trade\nPrice Per Unit\n2 (Rs) @,Brokerage per\nUnit\n(Rs),Net Rate\nper Unit\n(Rs),Closing Rate per\nUnit (only for\nDerivatives)\n(Rs.),Net Total (Before\nLevies)\n(Rs),Remar\nks,Securities\nTransaction Tax,Commodity\nTransaction Tax,SEBI turnover\nFees,Stamp Duty,Margin Amt. Dr.\nin Trading A/c
0,NSE,1.200000e+15,13:56:26,53049750.0,13:56:26,NIPPON IND ETF LIQUI,S,12.0,NaN,999.99,4.9999,994.9901,0.0,"-11,939.88",NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,Delivery Total. - Cr.,NaN,12.0,NaN,NaN,NaN,NaN,NaN,"11,939.88",NaN,NaN,NaN,NaN,NaN,NaN
2,NSE,NaN,NaN,NaN,NaN,[INTER ST GST ON CHG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.07,NaN,NaN,NaN,NaN,NaN,NaN
3,NSE,NaN,NaN,NaN,NaN,[INTER STATE GST],NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.80,NaN,NaN,NaN,NaN,NaN,NaN
4,NSE,NaN,NaN,NaN,NaN,[SEBI FEES],NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.01,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# import os
# from img2table.document import PDF
# from paddleocr import PaddleOCR
# import pandas as pd
# import numpy as np

# # Instantiation of the pdf
# pdf = PDF(src="/content/sample1.pdf")

# # Instantiation of the OCR, Tesseract, which requires prior installation
# ocr = PaddleOCR(lang="en")

# # Table identification and extraction
# pdf_tables = pdf.extract_tables(ocr=ocr, borderless_tables=True)

# # # We can also create an excel file with the tables
# pdf.to_xlsx('tables.xlsx',ocr=ocr)

[2023/02/25 18:41:20] ppocr DEBUG: Namespace(alpha=1.0, benchmark=False, beta=1.0, cls_batch_num=6, cls_image_shape='3, 48, 192', cls_model_dir='/root/.paddleocr/whl/cls/ch_ppocr_mobile_v2.0_cls_infer', cls_thresh=0.9, cpu_threads=10, crop_res_save_dir='./output', det=True, det_algorithm='DB', det_box_type='quad', det_db_box_thresh=0.6, det_db_score_mode='fast', det_db_thresh=0.3, det_db_unclip_ratio=1.5, det_east_cover_thresh=0.1, det_east_nms_thresh=0.2, det_east_score_thresh=0.8, det_limit_side_len=960, det_limit_type='max', det_model_dir='/root/.paddleocr/whl/det/en/en_PP-OCRv3_det_infer', det_pse_box_thresh=0.85, det_pse_min_area=16, det_pse_scale=1, det_pse_thresh=0, det_sast_nms_thresh=0.2, det_sast_score_thresh=0.5, draw_img_save_dir='./inference_results', drop_score=0.5, e2e_algorithm='PGNet', e2e_char_dict_path='./ppocr/utils/ic15_dict.txt', e2e_limit_side_len=768, e2e_limit_type='max', e2e_model_dir=None, e2e_pgnet_mode='fast', e2e_pgnet_score_thresh=0.5, e2e_pgnet_valid_set